In [1]:
import torch
import torch.nn as nn
import torchvision
import matplotlib.pyplot as plt
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
from torchvision import datasets,transforms

In [2]:
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.CenterCrop((500,281)),
                                transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225])])

target_transform = transforms.Compose([transforms.ToTensor(),
                                       transforms.CenterCrop((500,281)),])
train_set = datasets.VOCSegmentation(year='2007', image_set='trainval', root='../dataset',download=False,transform=transform,target_transform=target_transform)
test_set = datasets.VOCSegmentation(year='2007', image_set='test', root='../dataset',download=False,transform=transform,target_transform=target_transform)
train_loader = torch.utils.data.DataLoader(train_set, batch_size=8, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=8, shuffle=False)
print(len(train_loader))

53


In [3]:
class FCN(nn.Module):
    def __init__(self):
        super(FCN,self).__init__()
        self.block1 = nn.Sequential(
            nn.Conv2d(in_channels=3,out_channels=64,kernel_size=3,stride=1,padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2,2),
        )
        self.block2 = nn.Sequential(
            nn.Conv2d(in_channels=64,out_channels=128,kernel_size=3,stride=1,padding=1),
            nn.ReLU(),
            nn.AvgPool2d(2,2),
        )
        self.block3 = nn.Sequential(
            nn.Conv2d(in_channels=128,out_channels=256,kernel_size=3,stride=1,padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2,2),
        )
        self.block4 = nn.Sequential(
            nn.Conv2d(in_channels=256,out_channels=512,kernel_size=3,stride=1,padding=1),
            nn.ReLU(),
            nn.AvgPool2d(2,2),
        )
        self.transvonv = nn.Sequential(
            nn.ConvTranspose2d(in_channels=512,out_channels=256,kernel_size=2,stride=2),
            nn.ReLU(),

            nn.ConvTranspose2d(in_channels=256,out_channels=128,kernel_size=2,stride=2),
            nn.ReLU(),

            nn.ConvTranspose2d(in_channels=128,out_channels=64,kernel_size=2,stride=2),
            nn.ReLU(),

            nn.ConvTranspose2d(in_channels=64,out_channels=32,kernel_size=2,stride=2),
            nn.ReLU(),

            nn.ConvTranspose2d(in_channels=32,out_channels=16,kernel_size=(5,10),stride=1),
            nn.ReLU(),

            nn.Conv2d(in_channels=16,out_channels=21,kernel_size=1),
        )

    def forward(self, x):
        out = self.block1(x)
        out = self.block2(out)
        out = self.block3(out)
        out = self.block4(out)
        out = self.transvonv(out)
        return out

model = FCN()
def weights_init(m):
    if isinstance(m, nn.Conv2d):
        nn.init.xavier_uniform_(m.weight)
model.apply(weights_init)
model.to(device)

FCN(
  (block1): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (block2): Sequential(
    (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): AvgPool2d(kernel_size=2, stride=2, padding=0)
  )
  (block3): Sequential(
    (0): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (block4): Sequential(
    (0): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): AvgPool2d(kernel_size=2, stride=2, padding=0)
  )
  (transvonv): Sequential(
    (0): ConvTranspose2d(512, 256, kernel_size=(2, 2), stride=(2, 2))
    (1): ReLU()
    (2): ConvTranspose2d(256, 128, kernel_size=(2, 2), stride=(2, 2))
    (3): ReLU()
    (4): ConvTranspose2d(128, 64, kern

In [4]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [5]:

def train(num_epochs):
    model.train()
    losses = []
    epoches = list(torch.arange(1, num_epochs + 1))
    for epoch in range(num_epochs):
        running_loss = 0.0
        total = 0
        correct = 0
        for idx, (x, y) in enumerate(train_loader):
            optimizer.zero_grad()
            x, y = x.to(device), y.to(device).squeeze(1).long()
            y_pred = model(x)
            loss = criterion(y_pred, y)

            loss.backward()
            optimizer.step()
            running_loss += loss.item() * x.size(0)
            
            total += torch.numel(y)
            correct += (y_pred.argmax(dim=1) == y).sum().item()
            if (idx + 1) % 20 == 0:
                print(
                    f"Epoch [{epoch + 1}/{num_epochs}],Batches[{idx + 1}/{len(train_loader)}], Loss: {running_loss / ((idx + 1) * (x.size(0))):.3f} , Accuracy: {(100 * correct / total):.2f}%")
        losses.append(running_loss / len(train_loader.dataset))
        print(f"第{epoch + 1}轮的训练精度为：{(100 * correct / total):.2f}%\n")

    plt.plot(epoches, losses)
    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.show()

def test():
    model.eval()  # 设置模型为评估模式
    correct = 0
    total = 0

    with torch.no_grad():  # 不计算梯度
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device).squeeze(1).long()

            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)  # 获取预测结果
            total += torch.numel(labels)
            correct += (predicted == labels).sum().item()

    print(f'Test Accuracy: {100 * correct / total:.2f}%')




In [ ]:

    train(1)
    test()


Epoch [1/1],Batches[20/53], Loss: 1.399 , Accuracy: 71.73%
Epoch [1/1],Batches[40/53], Loss: 0.846 , Accuracy: 83.85%
第1轮的训练精度为：86.70%

